# Kruskal's Algorithms

Kruskal's algorithm is a greedy algorithm that is used to find the minimum spanning tree (MST) for a weighted, undirected graph.

## What is a minimum spanning tree?

Before, getting started with the algorithm, let us define, what is a minimum spanning tree?

Spanning trees are sub-graphs consisting of a subset of edges that connect all the nodes of the graph. For a given graph there could exist multiple spanning trees but there exists only one minimum spanning tree. The minimum spanning tree is defined as a sub-graph which contains the set of edges whose total weight is less than or equal to the weight of a spanning tree that covers all the nodes of the graph without forming any cycles. Simply put, a minimum spanning tree or MST for short is the collection of minium edges that will cover all the nodes on the graph.

The Minimum Spanning Trees are valid only for undirected graphs. For discussions of related concepts for directed graph, refer [minimum spanning arborescence](http://math.mit.edu/~goemans/18433S07/arborescence.pdf)


## Application of MST in Networking
In the context of networking, the minimum spanning tree can be used in **Network Design**. Consider the scenario when we are interested to connect different offices/ devices that are spread over an area over a network. In this case, we would want to optimize for minimizing the total cost for providing the connections. Thus, a MST is used,
>so that only the minimum number of packets need to be relayed across the network and multiple copies of the same packet don't arrive via different paths (remember, any two nodes are connected via only a single path in a spanning tree). [[7]](#)

Maximum Spanning Tree (Minimum Spanning Trees on negated edge weights) is used in problems like the **Widest Path Problem**. In the context of maximum flow in a network, the [widest path problem](https://en.wikipedia.org/wiki/Widest_path_problem) refers to a path which has the highest flow between a specified pair of source and target nodes. In the widest path for a given graph, the amount of flows is equivalent to the smallest weight of an edge on the path, this means that the smallest weight constrains the amount of flows that can be sent from the source to the target node.

Another application of MSTs in networking can be found in the spanning tree protocol, where the minimum spanning trees are used to avoid sending packets in cycles.

## Fetching required modules

Before, we get started lets import code modules which would make us focus on understanding the Kruskal's algorithm.

In [ ]:
import os, sys, subprocess
from os.path import dirname, join, abspath
import pkg_resources
import warnings
warnings.filterwarnings('ignore')

sys.path.insert(0, abspath(join(dirname("modules"), '..')))
required = {'numpy', 'matplotlib', 'networkx', 'graphviz'}
installed = {pkg.key for pkg in pkg_resources.working_set}
missing = required - installed

if missing:
  if 'google.colab' in str(get_ipython()):
    print('Running on CoLab')
    !git clone https://github.com/cni-iisc/networks-course.git
    !ln -sf networks-course/modules modules
  else:
    print('Not running on CoLab; please make sure to install the packages listed in requirements.txt')
  
from modules.create_graph import *
from modules.visualize_graph import *

<a class="anchor" id="graph1"></a>
## Building the graph

To establish the implementation of the Kruskal's algorithm, we create a weighted, undirected graph which is the same graph used in the previous notebooks.

In [ ]:
a = Node()
b = Node()
c = Node()
d = Node()
e = Node()
f = Node()

graphs = Graph.createGraph([a, b, c, d, e, f], directed=False)

graphs.add_Edge(a,b,1)
graphs.add_Edge(a,c,10)
graphs.add_Edge(a,e,2)
graphs.add_Edge(b,c,10)
graphs.add_Edge(b,d,6)
graphs.add_Edge(c,d,1)
graphs.add_Edge(c,f,10)
graphs.add_Edge(d,e,3)

visualizeGraph(graphs, "kruskal")

<a class="anchor" id="algorithm"></a>
## About the algorithm
The Kruskal Algorithm is a greedy algorithm to find the minimum spanning tree for an undirected, connected, weighted graph. If the term connected graph, sounds familiar, it is a graph where a path exists from one node to every other node on the graph. The algorithm has three primitive operations:
1. sort the edges of the graph in ascending order
2. pick the smallest edge and add it to the minimum spanning tree as long as it does not form a cycle
3. repeat operation (2) till the minimum spanning tree contains (|nodes in the graph| - 1) edges.

If you are wondering, how Kruskal's algorithm is greedy? 
The answer lies in operation (2) where in every iteration the algorithm picks the edge with the smallest weight for the minimum spanning tree. 

In operation (2), we select the edges for the minimum spanning tree such that the selected edge does not form a cycle. In this algorithm the Union-Find method also known as the [disjoint-set data strucutre](https://en.wikipedia.org/wiki/Disjoint-set_data_structure) is used. This operation is at the heart of the Kruskal's algorithm, so let us look into the union-find method. 

### Union-Find data structure
The Kruskal's algorithm uses the Union-Find data structure to find the next node that is to be added to the minimum spanning tree. Essentially under the hood, the union-find data strucutre is implemented through two helper functions called `Find()` and `Union()` which takes a list/ array as an input.

The prerequisite for the Union-Find data structure is the initialization step called `MakeSet()` in which we create two lists called `parent` and `rank` for all the nodes on the graph. We set all the elements in `rank` as $0$ and set that every node is its own `parent`. The `parent` list indicates which the node which becomes the reprsentative member of the set. For easier understanding, this notion of parent is similar the previous node idea used in Dijkstra's algorithm.

**`Find()`**
*Find(x)* tracks the chain of parent nodes in the tree till the root node is reached. The root node is defined as the node where the node is its own parent. The root node is the representative of the set to which the node *x* belongs to or can be *x* itself. We implement `Find(x)` based on the Path Compression where the tree is flattened such that every node always points to the root when *find* is used. 

<pre>1 <b>function</b> <i>Find</i>(x) <b>is</b>
2    <i>root</i>&nbsp;:= <i>x</i>
3    <b>while</b> root.parent ≠ <i>root</i>
4        <i>root</i>&nbsp;:= <i>root</i>.parent
5
6    <b>while</b> x.parent ≠ <i>root</i>
7        parent&nbsp;:= x.parent
8        x.parent&nbsp;:= root
9        <i>x</i>&nbsp;:= parent
10
11  <b>return</b> <i>root</i>
</pre>

**`Union()`**
*Union(x, y)* builds the spanning tree from the disjoint trees by first *finding* the roots of the trees that contains *x* and *y*. If the roots for *x* and *y* are distinct, the trees are combined by attaching either the root of *x* with the root of *y* (or) vice-versa. We implement Union *by rank* to avoid building the tree such that its height is same as the number of nodes in the tree.  

*Union by rank* in principle combines two trees based on their ranks. Initially, the rank of all the trees is set to 0. When we would like to combine two trees $T_{1}$  and $T_{2}$ which have their roots (or) parents as $x \in T_{1}$and $ y \in T_{2}$ using union by rank, we do the following:
- If $rank[x] = rank[y]$: select $y$ as the root of $x$ or vice-versa. This means the height of the tree increases by one node and the rank of $x$ increases by 1.
- If $rank[x] > rank[y]$ or $rank[x] < rank[y]$: select the root which has the higher rank. This means the height of the tree increases by one node and the rank of the root which has the lower rank increases by 1.

<pre>1 <b>function</b> <i>Union</i>(x, y) <b>is</b>
2   <i>xRoot</i>&nbsp;:= <i>Find</i>(<i>x</i>)
3   <i>yRoot</i>&nbsp;:= <i>Find</i>(<i>y</i>)
4 
5   <u>// x and y are already in the same set</u>
6   <b>if</b> <i>xRoot</i> = <i>yRoot</i> <b>then</b>
7       <b>return</b>
8 
9   <u>// x and y are not in same set, so we merge them</u>
10   <b>if</b> <i>xRoot</i>.rank &lt; <i>yRoot</i>.rank <b>then</b>
11     <i>xRoot</i>, <i>yRoot</i>&nbsp;:= <i>yRoot</i>, <i>xRoot</i> // swap xRoot and yRoot
12 
13   <u>// merge yRoot into xRoot</u>
14   yRoot.parent&nbsp;:= xRoot
15   <b>if</b> xRoot.rank = yRoot.rank <b>then</b>
16     xRoot.rank&nbsp;:= xRoot.rank + 1
</pre>




### Psuedo code of Kruskal's algorithm
<pre>1 <b>function</b> Kruskal(<i>G</i>) <b>is</b>
2    mstList&nbsp;:= []
3    parent&nbsp;:= []
4    <b>for each</b> v ∈ G.V <b>do</b>
5        parent.add(v) //make set of disjoint trees
6    <b>for each</b> (u, v) <b>in</b> G.E ordered by weight(u, v), increasing <b>do</b>
7        <b>if</b> FIND(parent, u) ≠ FIND(parent, v) <b>then</b>
8           mstList&nbsp;:= mstList ∪ {(u, v)}
9           UNION(FIND(parent, u), FIND(parent, v))
10    <b>return</b> mstList
</pre>

<a class="anchor" id="implementation"></a>
## Implementing Kruskal's algorithm

In [ ]:
#Function to find set(list implementation) of a node
def find(parent, nodeIndex):
    if parent[nodeIndex] == nodeIndex:
        return nodeIndex
    return find(parent, parent[nodeIndex])

#Function implementing union by rank for two sets x, y
def union(parent, rank, x, y):
    xRoot = find(parent, x)
    yRoot = find(parent, y)
    
    #Attach smaller rank under the root of a higher rank
    if rank[xRoot] < rank[yRoot]:
        parent[xRoot] = yRoot
    elif rank[xRoot] > rank[yRoot]:
        parent[yRoot] = xRoot
    #If ranks are same, pick one root and increment its rank by 1
    else:
        parent[yRoot] = xRoot #you could also make `xRoot` as root
        rank[yRoot] += 1
        
#Function implementing the Kruskal's algorithm
def kruskal(graph):
    mst = [] #stores the edges of the minimum spanning tree (MST)
    sortedListIndex = 0
    mstIndex = 0
    
    #sort the list of edges in the graph in ascending order
    edges = graph.get_allEdges()
    edges = sorted(edges, key=lambda item: item[2])
    
    parent = [] #previous nodes
    rank = []   #rank of the node
    
    #create a forest(group of trees) where each node is a tree = makeset operation
    for node in graph.get_allNodes():
        parent.append(node.index)
        rank.append(0)

    #number edges to be taken is |V| - 1
    while mstIndex < len(graph.get_allNodes()) - 1:
        #pick the edge with smallest weight and increment list index
        u, v, w = edges[sortedListIndex]
        sortedListIndex += 1
        
        x = find(parent, u)
        y = find(parent, v)
        
        #Add the edge to MST if it doesn't cause a cycle and increment mstIndex 
        if x != y:
            mstIndex += 1
            mst.append([u, v, w])
            union(parent, rank, x, y)
        #Else discard edge

    return mst

In [ ]:
mst = kruskal(graphs)
displayPath(mst, "kruskal")

<a class="anchor" id="proof"></a>
## Proof of correctness
In order to prove the correctness of the Kruskal's algorithm, we need to first prove that the algorithm yields a spanning tree and then prove that the spanning tree has minimum weight.


#### Proving a spanning tree is produced
Let $G$ be an undirected, connected, weighted graph and let $G^{*}$ be the sub-graph produced by Kruskal's algorithm. By definition of Kruskal's algorithm, the sub-graph $G^{*}$ produced does not have a cycle since the sub-graph is a single sub-tree.  The sub-graph $G^{*}$ cannot be disconnected, since the algorithm picks and adds the first minimum edge to connect two components of $G^{*}$. A component of graph refers to a sub-graph of a connected, undirected graph where there exists a path connecting two nodes on the sub-graph which does not connect to any additional nodes. This would mean that the sub-graph $G*$ is a spanning tree of $G$, since, the edges in $G^{*}$ connect all the nodes on the graph, $G$.

#### Minimality
Now, that we establish that Kruskal's algorithm yields a spanning tree, we shall now prove that the spanning tree is minimum through induction.

Let us consider $E^{'}$ as the list of edges chosen at any given iteration of the algorithm. Then, we want to prove that there exists some minmum spanning tree of $G$ that contains $E^{'}$.

In the begining of the algorithm, $E^{'}$ is empty and since $G$ is an connected, weighted graph it always will have a minimum spanning tree. 

Let us now assume that for some edge that is not last edge in the list $E^{'}$, there exists a minimum spanning tree $T$, which contains $E^{'}$.

If the algorithm selects the next edge $e_{1}$ which is already present in tree $T$, then tree formed by the edges in $E^{'}$ and edge $e_{1}$ yields a minimum spanning tree.

Else, if $e_{1}$ is not part of the tree $T$ then it implies that edges forming the tree $T$ along with edge $e_{1}$ contains a cycle $C$. The edges in cycle $C$ do not belong to the list of edges $E^{'}$, since by adding the edge $e_{1}$ to $E^{'}$ does not result in a cycle where adding `e` to $T$ yields in a cycle. Let $e_{2}$ be an edge  such that in  $e_{2} \in C$ and $e_{2} \not\in E^{'} + e_{1}$. Note, the edge $e_{2} \in T$ and has not been considered by the algorithm, which means that $\text{weight of } e_{2} \geq \text{weight of } e_{1}$. Then, $T - e_{2} + e_{1}$ is a tree where $\text{weight of } (T - e_{2} + e_{1}) \leq \text{weight of } T$. 

Hence, $T - e_{2} + e_{1}$ is a minimum spanning tree that contains the list of edges $E^{'}$ and the edge $e_{1}$. 

Thus by induction, we prove that the list of edges $E^{'}$ chosen by the algorithm yields a minimum spanning tree.

<a class="anchor" id="references"></a>
## Reference

- [1] Leiserson, Charles Eric, Ronald L. Rivest, Thomas H. Cormen, and Clifford Stein. Introduction to algorithms. Chapter 23. Vol. 6. Cambridge, MA: MIT press, 2001.
- [2] Kruskal's algorithm on [Wikipedia](https://en.wikipedia.org/wiki/Kruskal%27s_algorithm)
- [3] Article on Kruskal's algorithm on [Geeks for Geeks](https://www.geeksforgeeks.org/kruskals-minimum-spanning-tree-algorithm-greedy-algo-2/)
- [4] Applications of Minimum Spanning Trees - [StackOverflow](https://stackoverflow.com/questions/21661341/real-world-applications-where-spanning-tree-data-structure-is-used)
- [5] Minimum Spanning Trees Lecture Slides from [Princeton](https://algs4.cs.princeton.edu/lectures/43MinimumSpanningTrees-2x2.pdf)
- [6] Disjoint-set data structure on [Wikipedia](https://en.wikipedia.org/wiki/Disjoint-set_data_structure)
- [7] Application of MST from Alex Allian's article on [cprogramming.com](https://www.cprogramming.com/tutorial/computersciencetheory/mst.html)